<img src="https://www.th-koeln.de/img/logo.svg" style="float:right;" width="200">

# 12th exercise: <font color="#C70039">First Reinforcement Learning Game (*Frozen Lake*) using OpenAI Gym</font>
* Course: AML
* Lecturer: <a href="https://www.gernotheisenberg.de/">Gernot Heisenberg</a>
* Author of notebook: <a href="https://www.gernotheisenberg.de/">Gernot Heisenberg</a>. This notebook is based on the great post and notebook from [Rodolfo Mendes](https://morioh.com/p/18a96b9091d3).
* Student: Nicolas Rehbach
* Matriculation Number: 11133387
* Date:   19.12.2022

<img src="https://launchyourintelligentapphome.files.wordpress.com/2019/05/frozenlake_legended.png?w=531" style="float: center;" width="600">

---------------------------------
**GENERAL NOTE 1**: 
Please make sure you are reading the entire notebook, since it contains a lot of information on your tasks (e.g. regarding the set of certain paramaters or a specific computational trick), and the written mark downs as well as comments contain a lot of information on how things work together as a whole. 

**GENERAL NOTE 2**: 
* Please, when commenting source code, just use English language only. 
* When describing an observation please use English language, too.
* This applies to all exercises throughout this course.

---------------------------------

### <font color="ce33ff">DESCRIPTION</font>:

#### OpenAI Gym
In this exercise you will be using Python and OpenAI Gym to develop your reinforcement learning algorithm. The Gym library is a collection of environments that can be used freely with the reinforcement learning algorithms.

Gym has a ton of environments ranging from simple text based games to Atari games like Breakout and Space Invaders. The library is intuitive to use and simple to install. Just run **pip install gym** and you are good to go! The link to Gym's installation instructions, requirements, and documentation is included in the description. 

Further reading about OpenAI Gym is available under https://www.gymlibrary.dev/.
This notebook is based on this great post and notebook from [Rodolfo Mendes](https://morioh.com/p/18a96b9091d3).

#### Frozen Lake
This description of the game is copied directly from Gym's website. 

*Winter is coming. You and your friends were tossing around a frisbee at the park when you made a wild throw that left the frisbee out in the middle of the lake. The water is mostly frozen, but there are a few holes where the ice has melted. If you step into one of those holes, you'll fall into the freezing water and die (Game over). At this time, there's an international frisbee shortage, so it's absolutely imperative that you navigate across the lake and retrieve the disc. However, the ice is slippery, so you won't always move in the direction you intend. The surface is described using a grid like the following:*

* SFFF
* FHFH
* FFFH
* HFFG

This grid is your environment! S is your (the agent's) starting point and it's safe. F represents the frozen surface and is also safe. H represents a hole and if your agent steps in a hole in the middle of a frozen lake, the game is over because the agent dies. Finally, G represents the goal, which is the space on the grid where the frisbee is located.

The agent can navigate *left, right, up, down* and the episode ends when the agent reaches the goal or falls in a hole. It receives a reward of **1** if it reaches the goal and **0** otherwise.

Here is the summary:
<img src="https://github.com/gheisenberg/AML/blob/main/images/FrozenLake.States.Rewards.png?raw=1" style="float: center;" width="800">

---------------------------------

### <font color="FFC300">TASKS</font>:
The tasks that you need to work on within this notebook are always indicated below as bullet points. 
If a task is more challenging and consists of several steps, this is indicated as well. 
Make sure you have worked down the task list and commented your doings. 
This should be done by using markdown.<br> 
<font color=red>Make sure you don't forget to specify your name and your matriculation number in the notebook.</font>

**YOUR TASKS in this exercise are as follows**:
1. import the notebook to Google Colab or use your local machine.
2. make sure you specified you name and your matriculation number in the header below my name and date. 
    * set the date too and remove mine.
3. read the entire notebook carefully 
    * add comments whereever you feel it necessary for better understanding
    * run the notebook for the first time. 
4. install gym into your env!
5. You will train an agent to play the *Frozen Lake* game using Q-learning and you will get a playback of how the agent does after being trained.
6. Again the task: Your agent has to navigate the grid by staying on the frozen surface without falling into any holes until it reaches the frisbee. If it reaches the frisbee, it wins with a reward of plus one. If it falls in a hole, it loses and receives no points for the entire episode.
7. Your tasks are highlighted in the notebook (see below)
---------------------------------

### Imports 
import all important libs including gym

In [3]:
import numpy as np
import gym
import random
import time
from   IPython.display import clear_output

### Creating the Environment
For creating your environment, just call *gym.make()* and pass a string of the name of the environment you want to set up. 
All the environments with their corresponding names you can use here are available on Gym's website (see above).
With this *env* object, you are able to query for information about the environment, sample states and actions, retrieve rewards and have your agent navigate the frozen lake. That is all made available to you conveniently with Gym.

In [ ]:
env = gym.make("FrozenLake-v1")

In [ ]:
env.reset()

0

### Creating the Q-Table
Now, construct your Q-table, and initialize all the Q-values to zero for each state-action pair.
The number of rows in the table is equivalent to the size of the state space in the environment, and the number of columns is equivalent to the size of the action space (see above). You can get this information using *env.observation_space.n* and *env.action_space.n* as shown below in the code. Then, you can use this information to build the Q-table and initialize it with zeros.

In [ ]:
# Creating our q table which consists of the 
# action space (up, down, right, left)
action_space_size = env.action_space.n
# state space (our 16 possible fields (4x4))
state_space_size = env.observation_space.n

q_table = np.zeros((state_space_size, action_space_size))

In [ ]:
# Since our agent does not know anything, every state action pair is 0
print(q_table)

[[0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]]


### Initializing Q-Learning hyperparameters
Now, we're going to create and initialize all the parameters needed to implement the Q-learning algorithm.

First, with *num_episodes*, you define the total number of episodes you want the agent to play during training. Then, with *max_steps_per_episode*, you define a maximum number of steps that your agent is allowed to take within a single episode. So, if by the 100th step, the agent has not reached the frisbee or fallen through a hole, then the episode will terminate with the agent receiving zero points.

Next, you will set your *learning_rate* and your *discount_rate* as well, which was represented with the symbol (lambda) in the course slides (keyword: discounted return G_t).

Now, the last four parameters are all related to the exploration-exploitation dilemma with respect to the epsilon-greedy policy. You are initializing your *exploration_rate* to **1** and setting the *max_exploration_rate* to **1** and a *min_exploration_rate* to **0.01**. The *max* and *min* are just bounds to how large or small your exploration rate can be. Remember, the exploration rate was represented with the symbol (epsilon) when discussed in the course slides.

Lastly, you will set the *exploration_decay_rate* to **0.01** to determine the rate at which the *exploration_rate* will decay.

**YOUR <font color="FFC300">TASK</font> in this exercise is as follows** (point 7 from the task list above):

All of the above parameters can change!
Your task is to create a *testplan* and tune all parameters by yourself and observe how they influence and change the performance of the algorithm. 
Make notes! They will help you during the exam.

In [ ]:
num_episodes = 10000  # total number of episodes played during our training
max_steps_per_episode = 100 # maximum steps the agent takes while episode

learning_rate = 0.2 # learning rate epsilon
discount_rate = 0.99 # discount rate 

exploration_rate = 1 
max_exploration_rate = 1
min_exploration_rate = 0.001
exploration_decay_rate = 0.005

Create a list to hold all of the rewards you will get from each episode. 
By means of this you can observe how your game score changes over time.

In [ ]:
rewards_all_episodes = []

In the following code section, the entire Q-learning algorithm is implemented as discussed in detail in the AML course. 
When this code is executed, this is exactly where the training will take place. 
* The first for-loop contains everything that happens within a single episode. 
* The second nested loop contains everything that happens for a single time-step.

Read all the red comments, as they contain lots of important information on the implementation.

1st Loop:

For each episode, we reset the environment and adjust the reward.

2nd Loop:
- Our exploration rate threshhold is a random number from the uniform distribution
- if the exploration rate threshhold is larger than our exploration rate our agent will exploit the environmnt
- else our action is a random choice

Take new action:
- We call the step function which transitions us to a new state, reward, done and info

Update Q-table based on the Bellman equation

Set our current state to the new state and add the reward

Adjust the exploration rate, which is the min_exploraton rate defined by us - the max_exploration rate - min_exploration rate multiplied times - our exploration decay rate * the number of episodes

    exploration_rate = min_exploration_rate + (max_exploration_rate - min_exploration_rate) * np.exp(-exploration_decay_rate*episode)


In [ ]:
# Q-learning algorithm

# loop: for a single episode
for episode in range(num_episodes):
    # initialize 'new episode' parameters
    state = env.reset()
    ''' The done variable just keeps track of whether or not your episode is finished.
    Initialize it to False when first starting the episode and you will see later where 
    it will get updated to notify you when the episode is over.'''
    done = False
    
    ''' Keep track of the rewards within the current episode as well.
    Hence, set rewards_current_episode = 0 since you start 
    with no rewards at the beginning of each episode.'''
    rewards_current_episode = 0

    # nested loop: for a single time-step
    for step in range(max_steps_per_episode): 
        # Exploration-exploitation trade-off
        '''For each time-step within an episode set your exploration_rate_threshold 
        to a random number between 0 and 1. This will be used to determine whether 
        your agent will explore or exploit the environment in this time-step.'''
        exploration_rate_threshold = random.uniform(0, 1)
        if exploration_rate_threshold > exploration_rate:
            action = np.argmax(q_table[state,:]) 
        else:
            action = env.action_space.sample()

        # Take new action
        '''After action is chosen, take that action by calling step() on your env object and 
        pass your action to it. The function step() returns a tuple containing the new state, 
        the reward for the action you took, whether or not the action ended the episode and 
        diagnostic information regarding the environment (helpful for debugging).'''
        new_state, reward, done, info = env.step(action)

        # Update Q-table for Q(s,a)
        '''Compare this implementation with the equation in the course slides.'''
        q_table[state, action] = q_table[state, action] * (1 - learning_rate) + \
        learning_rate * (reward + discount_rate * np.max(q_table[new_state, :]))
        
        '''Set your current state to the new_state that was returned when taking the last action 
        and then update the rewards from your current episode by adding the reward you received 
        for your previous action.'''
        # Set new state
        state = new_state
        # Add new reward 
        rewards_current_episode += reward 
        '''Then, check to see if your last action ended the episode 
        (game over by agent stepping in a hole or reaching the goal)! 
        If the action did end the episode, then jump out of this loop and start the next episode.
        Otherwise, transition to the next time-step.'''
        if done == True: 
            break
            

    # Exploration rate decay
    '''Once an episode is finished, you need to update your exploration_rate using exponential decay, 
    which just means that the exploration rate decays at a rate proportional to its current value. 
    You can decay the exploration_rate using the formula above, which makes use of all the exploration 
    rate parameters that were defined above in the hyperparameter section.'''
    exploration_rate = min_exploration_rate + (max_exploration_rate - min_exploration_rate) * np.exp(-exploration_decay_rate*episode)
    
    # Add current episode reward to total rewards list and move on to the next episode
    rewards_all_episodes.append(rewards_current_episode)


### All episodes training completed
After all episodes are finished you now just calculate the average reward per thousand episodes from your list that contains the rewards for all episodes so that you can print it out and see how the rewards changed over time.

In [ ]:
# Calculate and print the average reward per thousand episodes
rewards_per_thousand_episodes = np.split(np.array(rewards_all_episodes),num_episodes/1000)
count = 1000

print("********Average reward per thousand episodes********\n")
for r in rewards_per_thousand_episodes:
    print(count, ": ", str(sum(r/1000)))
    count += 1000

********Average reward per thousand episodes********

1000 :  0.11500000000000009
2000 :  0.16800000000000012
3000 :  0.17200000000000013
4000 :  0.2450000000000002
5000 :  0.19200000000000014
6000 :  0.17600000000000013
7000 :  0.17400000000000013
8000 :  0.19300000000000014
9000 :  0.18000000000000013
10000 :  0.21500000000000016


### Interpretation

From this print, you can see that the average reward per thousand episodes did indeed progress over time. When the algorithm first started training, the first thousand episodes only averaged a reward of almost **0.18**, but by the time it got to its last thousand episodes, the reward drastically improved to almost **0.7**.

Let's take a second to understand how you can interpret these results. Your agent played **10000** episodes. At each time step within an episode, the agent received a reward of **1** if it reached the frisbee, otherwise, it received a reward of **0**. If the agent did indeed reach the frisbee, then the episode finished at that time-step.

Hence, that means for each episode, the total reward received by the agent for the entire episode is either **1** or **0**. So, for the first thousand episodes, you can interpret this score as meaning that **18%** of the time the agent received a reward of **1** and won the episode. And by the last thousand episodes from a total of **10000**, the agent was winning almost **70%** of the time.

By analyzing the grid of the game, you can see it is a lot more likely that the agent would fall in a hole or perhaps reach the max time steps than it is to reach the frisbee, so reaching the frisbee **70%** of the time is not too bad, especially since the agent had no explicit instructions to reach the frisbee. It learned that this is the correct thing to do.

* SFFF
* FHFH
* FFFH
* HFFG

At last, print out your updated Q-table to see how that has transitioned from its initial state of all zeros.

In [ ]:
# Print updated Q-table
print("\n\n********Q-table********\n")
print(q_table)



********Q-table********

[[0.11658061 0.0554133  0.10225941 0.10106414]
 [0.02070561 0.02815919 0.02063497 0.07455307]
 [0.04840272 0.01470083 0.01657251 0.01755061]
 [0.01243346 0.01239125 0.00749147 0.03108224]
 [0.14382438 0.0395896  0.02108416 0.03687206]
 [0.         0.         0.         0.        ]
 [0.21830464 0.02810786 0.00338817 0.00344378]
 [0.         0.         0.         0.        ]
 [0.13257464 0.10605136 0.13856745 0.42008916]
 [0.28198252 0.51886139 0.10327252 0.21660502]
 [0.66192755 0.03303607 0.04343606 0.01257658]
 [0.         0.         0.         0.        ]
 [0.         0.         0.         0.        ]
 [0.12461402 0.09829155 0.67990866 0.10151444]
 [0.32530198 0.97067467 0.49908067 0.21608362]
 [0.         0.         0.         0.        ]]


## Experimenting with different Hyperparameters

In [33]:
def run_experiment(num_episodes, max_steps_per_episode, learning_rate, discount_rate, exploration_rate, max_exploration_rate, min_exploration_rate, exploration_decay_rate):
  env = gym.make("FrozenLake-v1")
  env.reset()

  action_space_size = env.action_space.n
  # state space (our 16 possible fields (4x4))
  state_space_size = env.observation_space.n

  q_table = np.zeros((state_space_size, action_space_size))
  
  rewards_all_episodes = []

  # Q-learning algorithm

  # loop: for a single episode
  for episode in range(num_episodes):
    # initialize 'new episode' parameters
    state = env.reset()
    ''' The done variable just keeps track of whether or not your episode is finished.
    Initialize it to False when first starting the episode and you will see later where 
    it will get updated to notify you when the episode is over.'''
    done = False
    
    ''' Keep track of the rewards within the current episode as well.
    Hence, set rewards_current_episode = 0 since you start 
    with no rewards at the beginning of each episode.'''
    rewards_current_episode = 0

    # nested loop: for a single time-step
    for step in range(max_steps_per_episode): 
        # Exploration-exploitation trade-off
        '''For each time-step within an episode set your exploration_rate_threshold 
        to a random number between 0 and 1. This will be used to determine whether 
        your agent will explore or exploit the environment in this time-step.'''
        exploration_rate_threshold = random.uniform(0, 1)
        if exploration_rate_threshold > exploration_rate:
            action = np.argmax(q_table[state,:]) 
        else:
            action = env.action_space.sample()

        # Take new action
        '''After action is chosen, take that action by calling step() on your env object and 
        pass your action to it. The function step() returns a tuple containing the new state, 
        the reward for the action you took, whether or not the action ended the episode and 
        diagnostic information regarding the environment (helpful for debugging).'''
        new_state, reward, done, info = env.step(action)

        # Update Q-table for Q(s,a)
        '''Compare this implementation with the equation in the course slides.'''
        q_table[state, action] = q_table[state, action] * (1 - learning_rate) + \
        learning_rate * (reward + discount_rate * np.max(q_table[new_state, :]))
        
        '''Set your current state to the new_state that was returned when taking the last action 
        and then update the rewards from your current episode by adding the reward you received 
        for your previous action.'''
        # Set new state
        state = new_state
        # Add new reward 
        rewards_current_episode += reward 
        '''Then, check to see if your last action ended the episode 
        (game over by agent stepping in a hole or reaching the goal)! 
        If the action did end the episode, then jump out of this loop and start the next episode.
        Otherwise, transition to the next time-step.'''
        if done == True: 
            break
            

    # Exploration rate decay
    '''Once an episode is finished, you need to update your exploration_rate using exponential decay, 
    which just means that the exploration rate decays at a rate proportional to its current value. 
    You can decay the exploration_rate using the formula above, which makes use of all the exploration 
    rate parameters that were defined above in the hyperparameter section.'''
    exploration_rate = min_exploration_rate + (max_exploration_rate - min_exploration_rate) * np.exp(-exploration_decay_rate*episode)
    
    # Add current episode reward to total rewards list and move on to the next episode
    rewards_all_episodes.append(rewards_current_episode)

  rewards_per_thousand_episodes = np.split(np.array(rewards_all_episodes),num_episodes/1000)
  count = 1000

  print("********Average reward per thousand episodes********\n")
  for r in rewards_per_thousand_episodes:
    print(count, ": ", str(sum(r/1000)))
    count += 1000 
  print("\n\n********Final Q-table********\n")
  print(q_table)

In [38]:
#num_episodes, max_steps_per_episode, learning_rate, discount_rate, exploration_rate, max_exploration_rate, min_exploration_rate, exploration_decay_rate
run_experiment(50000, 500, 0.2, 0.99, 1, 1, 0.001, 0.005)

********Average reward per thousand episodes********

1000 :  0.2650000000000002
2000 :  0.6960000000000005
3000 :  0.7200000000000005
4000 :  0.7300000000000005
5000 :  0.7320000000000005
6000 :  0.7160000000000005
7000 :  0.6930000000000005
8000 :  0.7310000000000005
9000 :  0.7110000000000005
10000 :  0.7100000000000005
11000 :  0.7110000000000005
12000 :  0.7020000000000005
13000 :  0.7430000000000005
14000 :  0.7110000000000005
15000 :  0.7270000000000005
16000 :  0.7170000000000005
17000 :  0.7270000000000005
18000 :  0.7080000000000005
19000 :  0.7040000000000005
20000 :  0.7430000000000005
21000 :  0.7230000000000005
22000 :  0.7140000000000005
23000 :  0.7160000000000005
24000 :  0.7490000000000006
25000 :  0.7160000000000005
26000 :  0.7120000000000005
27000 :  0.7100000000000005
28000 :  0.7160000000000005
29000 :  0.7260000000000005
30000 :  0.6990000000000005
31000 :  0.7230000000000005
32000 :  0.7130000000000005
33000 :  0.7400000000000005
34000 :  0.7070000000000005
350

### Testplan to optimize the algorithm:

We are working with the Q-algorithm to train our agent to reach the frisbee and learn from its mistakes. To calculate the new Q-value we use the formula:

$q^{new}(s,a) = (1-\alpha)q(s,a)+\alpha[R_{t+1}+\gamma max q (s', a')]$

with: 

$\alpha$ our learning rate, $\gamma$ our discount rate and $R_{t+1}$ being the reward.


Influence of variables (according to wikipedia)

Learning rate $\alpha$:
- Determines to what extent newly acquired information overrides the old one.
- 0 learns nothing (only using prior knowledge)
- 1 makes the agent only consider the most recent information (ignoring prior knowledge to explore possibilities)
- Fully deterministing -> alpha of 1 is optimal
- Stochastic -> algorithm converges so it decreases to zero
- In practice, a constant LR is used such as 0.1

Discount factor $\gamma$
- Determines the importance of future rewards
- Factor of 0 makes the agent "myopic" only short sighted and not considerent long term rewards
- Factor of >= 1 makes the rewards become infinite

Open Questions:
- Why does a reward of 0 happen?

Working out good Hyperparameters:

First, differences in the learning rate and the discount rate are going to be observed. Personally, I think the discount rate should not be lowered too much, since we do not want to have an short sighted agent. The learning rate on the other side could be important.

### Test 1: Base Hyperparameter

num_episodes = 10000, max_steps_per_episode = 100, learning_rate = 0.1, discount_rate = 0.99, exploration_rate = 1, max_exploration_rate = 1, min_exploration_rate = 0.01, exploration_decay_rate = 0.01

**Highest 1000 episodes value: 59%**

Interestingly, the first run was all 0s, after restarting the kernel and run all, a increasing reward percentage could be observed.

### Test 2: Increasing learning rate

num_episodes = 10000, max_steps_per_episode = 100, learning_rate = 0.2,
discount_rate = 0.99, exploration_rate = 1, max_exploration_rate = 1, min_exploration_rate = 0.01, exploration_decay_rate = 0.01

**Highest 1000 episodes value: 68.7%**

Because of the higher learning rate, the agent directly started with 51% in the first 1000 episodes. We can observe, that an higher learning rate led to generally higher average reward values.

### Test 3: Increasing learning rate

learning_rate = 0.3, discount_rate = 0.99, exploration_rate = 1, max_exploration_rate = 1, min_exploration_rate = 0.01, exploration_decay_rate = 0.01

**Highest 1000 episodes value: 68.7%**

The agent did not better his max average percentage. Overall all values converged towards 60-70 percent. However, a higher learning curve does not seem to help further improvement.

### Test 4: Increased learning rate, lowered discount rate

learning_rate = 0.2, discount_rate = 0.9, exploration_rate = 1, max_exploration_rate = 1, min_exploration_rate = 0.01, exploration_decay_rate = 0.01

**Highest 1000 episodes value: 52%**

We can observe, that lowering the discount rate does not help the algorithm, which was expected since we dont want short sighted agents. A increased learning rate, high discount rate and lower exploration decay could be interesting.

### Test 5: Increased learning rate, same discount rate, lower exploratory decay

learning_rate = 0.2, discount_rate = 0.99 , exploration_rate = 1, max_exploration_rate = 1, min_exploration_rate = 0.01, exploration_decay_rate = 0.005

**Highest 1000 episodes value: 68%**

The outcome is quite similar to Test 2 and 3, maybe we have to have to change up the min_exploration rate in order to keep the agent trying out longer.

### Test 6: Increased learning rate, ame discount rate, lower exploratory decay, lower min_exploration rate

num_episodes = 10000, max_steps_per_episode = 100, learning_rate = 0.2,  discount_rate = 0.99, exploration_rate = 1, max_exploration_rate = 1, min_exploration_rate = 0.001, exploration_decay_rate = 0.005

**Highest 1000 episodes value: 74%**

With this setup, we could break through the 70% mark. Possibly lower min exploration rates and lower exploration decays are key.



## Final observations:
- Adding more steps to the episodes did not help. 
- Changing the exploration_rate from 1 to 0.9 did not help
- Adding 100000 episodes did not help
- Generally a learning rate of 0.2 was optimal for our approach, higher ones resulted in worse results
- Changing the $\epsilon$-greedy parameters helped the agent
  - lower exploration decay rate and lowe min exploration rate helped out -> more exploiting
- 74-76% as barrier was not surpassed by trying out other hyperparameters.